In [18]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator as imgdatgen

Feel free to adjust the constants to suit your needs

In [14]:
IMG_SIZE = (80,80)
BATCH_SIZE = 32
EPOCH = 10
channels = 1
dataset_path = r"dataset"

Creating a frequency Dictionary to see how many images are there in each category

In [ ]:
chars = {}
for folder in os.listdir(dataset_path):
    chars[folder] = len(os.listdir(os.path.join(dataset_path, folder)))
chars_list= caer.sort_dict(chars , descending=True)

If there are more a large number of categories, You can choose to train a small amount of them with.
<br>The entire code is assuming you train only the top 10 categories with highest number of samples.
<br>You can choose to skip it and alter the code to suit you needs.
<br>You can replace the variable <span style='color: orange;'>top</span> with variable <span style='color: orange;'>chars_list</span>.
<br>if you wish to train all the images in the dataset

In [ ]:
top = [i[0] for i in chars_list[0:10]]

Creating a training set from the data of Top 10 categories
<br>If there are more categories, It will take more time depending upon the processing power of your computer

In [ ]:
training_set = caer.preprocess_from_dir(dataset_path,top, channels = channels,IMG_SIZE=IMG_SIZE,isShuffle=True)

In [ ]:
featureSet , label = caer.sep_train(training_set,IMG_SIZE=IMG_SIZE)

Normalize feature set and change labels into categorical values

In [ ]:
featureSet = caer.normalize(featureSet)
label = to_categorical(label,len(top))

splitting the data into training set and validation set(testing set)

In [ ]:
x_train , x_val , y_train , y_val = caer.train_val_split(featureSet,label,val_ratio=0.25)

the training_set , featureSet and labels datas are useless at this point since we created a training and testing dataset and you can garbage collect them whenever you want after this point to reduce memory useag using the command

<br> <span style='color: red;'>del</span> training_set
<br> <span style='color: red;'>del</span> featureSet
<br> <span style='color: red;'>del</span> label
<br> gc.<span style='color: green;'>collect()</span>


In [ ]:
datagen = canaro.generators.imageDataGenerator()

<span style='color: yellow;'> canaro.generators.imageDataGenerator()</span>
<br> is just imageDataGenerator of Tensorflow with certain arguements already passed
<br>please feel free to modify the dataget variable with ImageDataGenerator() and give your own arguments to match your needs
<br>You can change it bu doing
<br><span style='color: orange;'>from tensorflow.keras.preprocessing.image import ImageDataGenerator as imgdatgen</span>
<br>datagen = imgdatgen(<span style='color: orange;'>"Your Arguments Here"</span>)
<br>Refer Next CodeBlock for modifying imgdatagen parameters

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as imgdatgen
def imageDataGenerator():
    datagen = imgdatgen(rotation_range=10, 
                           width_shift_range=.1,
                           height_shift_range=.1,
                           shear_range=.2,
                           zoom_range=.2,
                           horizontal_flip=True,
                           fill_mode='nearest')
    return datagen

flow Takes data & label arrays, generates batches of augmented data.

In [ ]:
train_gen = datagen.flow(x_train,y_train,batch_size=BATCH_SIZE)

This is the most important part, This is the neural Network consiting of different layers
<br>Feel free to adjut data values as you see fit to match your usecase
<br>This is the Abstracted model of canaro.models.createSimpsonsModel() , you can use that directly if you dont want to play around with neural layers and
<br>Keras depreciated decay , change it to your needs

In [ ]:
def createModel(IMG_SIZE=(224,224), channels=1, output_dim=1, loss='binary_crossentropy', decay=None, learning_rate=None, momentum=None, nesterov=None):
    if not isinstance(output_dim, int):
        raise ValueError('[ERROR] Output dimensions need to be an integer')
    if not isinstance(channels, int):
        raise ValueError('[ERROR] Channels needs to be an integer')
        
    w, h = IMG_SIZE[:2]
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(w, h, channels)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu')) 
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(w, h,channels)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu')) 
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    
    # Output Layer
    model.add(Dense(output_dim, activation='softmax'))

    optimizer = SGD(learning_rate=learning_rate, decay=decay, momentum=momentum, nesterov=nesterov)

    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = createModel(IMG_SIZE=IMG_SIZE, channels = channels, 
                                        output_dim=len(top), 
                                        loss = 'binary_crossentropy',
                                        decay=1e-6, 
                                        learning_rate=0.001,
                                        momentum = 0.9,
                                        nesterov =True)

Schedule Learning Rate

In [ ]:
callback = [LearningRateScheduler(canaro.lr_schedule)]

Train model, increase/Decrease Epochs as you see fit

In [ ]:
training = model.fit(train_gen,
                    steps_per_epoch = len(train_gen) // BATCH_SIZE,
                    epochs = 20,
                    validation_data=(x_val,y_val),
                    validation_steps = len(y_val)//BATCH_SIZE)

Model is trained

Testing part, Ignore if you dont want to test individual images

In [ ]:
test_path = r"Path of the image"
img = cv.imread(test_path)

Converting images to suit the models input

In [ ]:
def conv(img):
    
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img = cv.resize(img,IMG_SIZE)
    img = caer.reshape(img,IMG_SIZE,1)
    return img

Prediction

In [ ]:
prediction = model.predict(conv(img))

Result

In [ ]:
print(top[np.argmax(prediction[0])])

To test every image in a given category of images and see how many are guessed correctly

In [ ]:
dicts = {}
faults = []
Dir = "folder Path of a category of images you used in training"
for i in os.listdir(Dir):
    try:
        imgs = Dir + i
        img = cv.imread(imgs)
        prediction = model.predict(conv(img))
        if top[np.argmax(prediction[0])] in dicts:
            dicts[top[np.argmax(prediction[0])]] += 1
        else:
            dicts[top[np.argmax(prediction[0])]] = 1
    except:
        faults.append(i)
        
        continue
print(dicts)
print(faults)
print(len(faults))